#【練習說明】
在做完 House Price 的預測中，我們作了補值與文字轉數值，送出了我們第一個 Kaggle 成績，
現在讓我們用同樣的作法，來完成經典鐵達尼號生存猜測問題。

Kaggle 鐵達尼練習題 https://www.kaggle.com/c/titanic/data
1. 進入上方往頁後, 下載訓練資料 train.csv 與 test.csv
2. 在自己的google drive中創建一個名為kaggle的資料夾, 將 train.csv 與 test.csv 上傳到此資料夾中
2. 依序執行下方程式後, 即可產生預測檔 titanic_baseline.csv, 經由 https://www.kaggle.com/c/titanic/submit 提交預測
3. 替換最後兩個區塊 (更改程式碼, 使用課程中所交過的模型), 試著產生不同的預測結果, 並比較分數的變化

# 0. 連結 Google雲端硬碟

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

kaggle_dir = 'drive/My Drive/kaggle/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


# 1. 載入所需套件

In [0]:
# 載入會使用到的套件
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# 2. 載入資料集，觀察資料集

In [0]:
# 利用 pd.read_csv 讀入我們所需的檔案, 參數後面是路徑
# 這邊利用我們先前設定好的資料夾目錄+檔名,就可以得到一個完整的路徑

titanic_train = pd.read_csv(kaggle_dir + 'train.csv')
titanic_test = pd.read_csv(kaggle_dir + 'test.csv')

# 觀察訓練集和測試集的維度
print(f'訓練資料的筆數與特徵值數目：{titanic_train.shape}')
print(f'測試資料的筆數與特徵值數目：{titanic_test.shape}')

訓練資料的筆數與特徵值數目：(891, 12)
測試資料的筆數與特徵值數目：(418, 11)


In [0]:
# 檢視訓練集的前五筆資料
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
# 檢視測試集的前五筆資料
titanic_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# 3. 資料前處理
- 這裡有兩個重點：
  - 補缺失值：缺失值在電腦中也是無法進行四則運算的，所以我們要補缺失值
  - 轉形態：電腦在字串之間進行數學的四則運算，所以要將字串轉換成數字
  

## 3-1. 補缺失值

In [0]:
# 那麼我們先處理缺失值的部分
# 我們可以利用以下指令把缺失值統計出來
print(titanic_train.isna().sum())
print("----")
print(titanic_test.isna().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
----
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [0]:
# 首先, 我們針對資料中我們熟悉的部分做補值
# Age 欄位是指乘客的年齡, 我們如果沒有比較好的補法, 可以使用捨去的方式, 或是補上中位數, 平均值等等的方法
# 這邊我們先以補平均值的方法

titanic_train.Age.fillna( titanic_train.Age.mean(), inplace=True)

# 那麼要記得訓練集做了什麼前處理, 測試集也要跟著做一樣的動作
# 所以測試集也要補上平均值

titanic_test.Age.fillna( titanic_test.Age.mean(), inplace=True)

In [0]:
# Cabin 的缺失值太多, 將近佔了整體資料的八成, 我們可以大膽的先捨去這個欄位
titanic_train.drop(columns='Cabin', inplace=True)

# 一樣訓練集做了什麼, 測試集也要跟著做
titanic_test.drop(columns='Cabin', inplace=True)

In [0]:
# 接著是 Embarked, 從上方可以看到只有2筆缺失, 我們可以練習使用眾數來補值
titanic_train.Embarked.fillna( titanic_train.Embarked.mode()[0], inplace=True)

# 一樣訓練集做了什麼, 測試集也要跟著做
titanic_test.Embarked.fillna( titanic_test.Embarked.mode()[0], inplace=True)

In [0]:
# 測試集的 Fare有缺失值, 但是訓練集沒有, 所以我們這邊針對測試集做補值
# 使用平均值補值
titanic_test.Fare.fillna( titanic_test.Fare.mean() , inplace=True)

In [0]:
# 接著再次檢查是不是還有缺失值
print(titanic_train.isna().sum())
print("-----")
print(titanic_test.isna().sum())

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64
-----
PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


## 3-2. 轉型態

In [0]:
# 先看一下資料的型態 
# 我們要將 Object 型態的資料都轉換成 int或 float, 讓電腦能夠進行四則運算
print(titanic_train.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Embarked        object
dtype: object


In [0]:
# 沒有直觀轉換的方式我們可以選擇直接捨去, 例如 Name及 Ticket, 幾乎都是不重複的字串
# 所以我們就直接捨去這兩個欄位

titanic_train.drop(columns=['Name','Ticket'], inplace=True)
titanic_test.drop(columns=['Name','Ticket'], inplace=True)

In [0]:
# 接著, 還剩下 Sex及 Embarked要做轉換
# 我們可以使用 sklearn 的 LabelEncoder 工具來幫我們做 label encoding

# 首先先取出套件，存到變數 le中
le = LabelEncoder()
titanic_train.Sex = le.fit_transform(titanic_train.Sex)
# 測試集也要跟著做, 但要注意這邊是 transform 指令, 因為在訓練集已經讓工具知道對應關係了, 這邊直接做轉換即可
titanic_test.Sex = le.transform(titanic_test.Sex)

# 接著做 Embarked的轉換
titanic_train.Embarked = le.fit_transform(titanic_train.Embarked)
titanic_test.Embarked = le.transform(titanic_test.Embarked)


In [0]:
# 最後再次檢查是不是都是 int或 float型態
print(titanic_train.dtypes)
print("----")
print(titanic_test.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Embarked         int64
dtype: object
----
PassengerId      int64
Pclass           int64
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Embarked         int64
dtype: object


# 4. 準備訓練集及測試集

In [0]:
# 我們再次把訓練資料給印出來, 以便我們選擇欄位進入模型訓練
# 可以看到 PassengerId 是每一列獨一無二的編號, 是可以捨去不進行訓練的
# 而 Survived 欄位其實就是我們要去訓練的 y
titanic_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,2
1,2,1,1,0,38.0,1,0,71.2833,0
2,3,1,3,0,26.0,0,0,7.9250,2
3,4,1,1,0,35.0,1,0,53.1000,2
4,5,0,3,1,35.0,0,0,8.0500,2


In [0]:
# 準備訓練集與測試集

# train_x 不需要這兩個欄位
train_x = titanic_train.drop(columns=['PassengerId', 'Survived'])

# train_y 就是 Survived欄位
train_y = titanic_train['Survived']

# test_x 的規則與 train_x 相同, 但是測試集本身就沒有 Survived的欄位, 所以不需要捨棄
test_x = titanic_test.drop(columns=['PassengerId'])

# 4. 訓練模型

In [0]:
# 這邊我們選擇羅吉斯回歸模型
from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()

# 進行模型訓練
estimator.fit(train_x, train_y)

# 進行模型預測
pred = estimator.predict(test_x)

In [0]:
# 提交檔生成
# 提交檔需要兩個欄位, 及測試集的 PassengerId, 以及每個 Id 對應到的 Survived, 即我們的預測值
submit = pd.DataFrame({'PassengerId': titanic_test.PassengerId, 'Survived': pred})

# 最後就可以存成 csv 檔案, 提交至 Kaggle !
submit.to_csv(kaggle_dir +'titanic_baseline.csv', index=False) 